In [1]:
import cv2
import os

In [2]:
main_dir = os.getcwd()
main_dir = os.path.join(main_dir, 'drive')
main_dir = os.path.join(main_dir, 'MyDrive')
main_dir = os.path.join(main_dir, 'Colab Notebooks')

train_dir = os.path.join(main_dir, 'tr')
test_dir = os.path.join(main_dir, 'te')
val_dir = os.path.join(main_dir, 'va')

In [3]:
from keras.applications import vgg16

In [4]:
img_rows, img_cols = 224, 224

model = vgg16.VGG16(weights = 'imagenet',
                    include_top = False,
                    input_shape = (img_rows, img_cols, 3))

for layer in model.layers:
  layer.trainable = False

58900480/58889256 [==============================] - 0s 0us/step


In [5]:
def layer_adder(bottom_model, num_classes):
  top_model = bottom_model.output
  top_model = GlobalAveragePooling2D()(top_model)
  top_model = Dense(1024, activation = 'relu')(top_model)
  top_model = Dense(512, activation = 'relu')(top_model)
  top_model = Dense(num_classes, activation = 'softmax')(top_model)
  return top_model

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

num_classes = 10

FC_head = layer_adder(model, num_classes)

model = Model(inputs = model.input, outputs = FC_head)

print(model.summary)

<bound method Model.summary of <keras.engine.functional.Functional object at 0x7f507e3246d0>>


In [7]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255)

val_datagen = ImageDataGenerator(rescale = 1./255)

batch_size = 16

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (img_rows, img_cols),
    batch_size = batch_size,
    class_mode = 'categorical')

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size = (img_rows, img_cols),
    batch_size = batch_size,
    class_mode = 'categorical')

Found 1174 images belonging to 10 classes.
Found 361 images belonging to 10 classes.


In [8]:
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("face_detector.h15",
                             monitor = "val_loss",
                             mode = "min",
                             save_best_only = True,
                             verbose = 1)

earlystop = EarlyStopping(monitor = "val_loss",
                          min_delta = 0,
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

callbacks = [earlystop, checkpoint]

model.compile(loss = 'categorical_crossentropy',
              optimizer = 'Adam',
              metrics = 'accuracy')

nb_train_samples = 1174 
nb_val_samples = 361 

epochs = 10
batch_size = 16

history = model.fit_generator(
    train_generator,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = val_generator,
)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
74/74 [==============================] - 463s 6s/step - loss: 2.2515 - accuracy: 0.1514 - val_loss: 1.7559 - val_accuracy: 0.3380

Epoch 00001: val_loss improved from inf to 1.75593, saving model to face_detector.h15
INFO:tensorflow:Assets written to: face_detector.h15/assets
Epoch 2/10
74/74 [==============================] - 6s 86ms/step - loss: 1.6671 - accuracy: 0.4002 - val_loss: 1.4317 - val_accuracy: 0.5125

Epoch 00002: val_loss improved from 1.75593 to 1.43165, saving model to face_detector.h15
INFO:tensorflow:Assets written to: face_detector.h15/assets
Epoch 3/10
74/74 [==============================] - 6s 86ms/step - loss: 1.3821 - accuracy: 0.5075 - val_loss: 1.3449 - val_accuracy: 0.5125

Epoch 00003: val_loss improved from 1.43165 to 1.34486, saving model to face_detector.h15
INFO:tensorflow:Assets written to: face_detector.h15/assets
Epoch 4/10
74/74 [==============================] - 6s 85ms/step - loss: 1.2122 - accuracy: 0.5420 - val_loss: 1.1840 - val_accu